# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from functools import reduce
import matplotlib
from past.builtins import xrange
from textwrap import wrap
from matplotlib import cm
import random
import matplotlib.colors as mcolors
import seaborn as sns
from matplotlib.patches import Rectangle
import pickle

In [2]:
prefix   = 'E:/ML Fire/EPRI-Data/EPRI_OG/EPRI-FEDB_DA/'
plot     = 'Plots/'
old      = 'Old/'
updated_data = 'Updated Data/'
up_data_plot =  'Updated Data/Plots/'
sys_plot = os.path.join(prefix, plot)
sys_old  = os.path.join(prefix, old)
sys_up_data = os.path.join(prefix, updated_data)
sys_up_dplo = os.path.join(prefix, up_data_plot)

# Import EPRI data

**1990-2009**

In [3]:
EPRI_90_09 = 'EPRI.xlsx'

# Extracting Data-frames 
FES_o = pd.read_excel(EPRI_90_09, sheet_name = 'Fire Event Summaries')
FEA_o = pd.read_excel(EPRI_90_09, sheet_name = 'Fire Event Attributes')
FSE_o = pd.read_excel(EPRI_90_09, sheet_name = 'Fire Severity Evaluation')
FTS_o = pd.read_excel(EPRI_90_09, sheet_name = 'Fire Timeline and Suppression')
PR_o  = pd.read_excel(EPRI_90_09, sheet_name = 'Plant Response')

epri_o = [FES_o, FEA_o, FSE_o, FTS_o, PR_o]

In [4]:
print(FES_o.shape, FEA_o.shape, FSE_o.shape, FTS_o.shape, PR_o.shape)

(2186, 11) (1972, 12) (2190, 7) (1816, 14) (1976, 9)


**2010-2014**

In [ ]:
EPRI_10_14 = 'EPRI_10_14.xlsx'

# Extracting Data-frames 
FES_n = pd.read_excel(EPRI_10_14, sheet_name = 'Fire Event Summaries')
FEA_n = pd.read_excel(EPRI_10_14, sheet_name = 'Fire Event Attributes')
FSE_n = pd.read_excel(EPRI_10_14, sheet_name = 'Fire Severity Evaluation')
FTS_n = pd.read_excel(EPRI_10_14, sheet_name = 'Fire Timeline and Suppression')
PR_n  = pd.read_excel(EPRI_10_14, sheet_name = 'Plant Response')

epri_n = [FES_n, FEA_n, FSE_n, FTS_n, PR_n]

In [ ]:
print(FES_n.shape, FEA_n.shape, FSE_n.shape, FTS_n.shape, PR_n.shape)

**Appending Sheets**

In [ ]:
FES_up = pd.concat([FES_o, FES_n], ignore_index=True)
FEA_up = pd.concat([FEA_o, FEA_n], ignore_index = True)
FSE_up = pd.concat([FSE_o, FSE_n], ignore_index = True)
FTS_up = pd.concat([FTS_o, FTS_n], ignore_index = True)
PR_up  = pd.concat([PR_o, PR_n], ignore_index = True)
EPRI_uplist = [FES_up, FEA_up, FSE_up, FTS_up, PR_up]

**Merging Sheets**

In [ ]:
EPRI_up = reduce(lambda x,y: pd.merge(x,y, on='FireID', how='outer', sort = True), EPRI_uplist)

# Data Cleanup

**Disposition**

In [ ]:
EPRI_up['Disposition_FES'].unique()

In [ ]:
EPRI_up['Disposition_FES'] = EPRI_up['Disposition_FES'].replace(['Potentially\xa0Challenging', ' Potentially\xa0Challenging', 'Potentailly Challenging ',
                                                      ' Not\xa0Challenging', 'Not\xa0Challenging',
                                                       'Undetermined\xa0(NC‐PC)', 'Undetermined\xa0(PC‐CH)', ' Undetermined\xa0(PC‐CH)', ' Undetermined\xa0(NC‐PC)',
                                                      'Undetermined(NC‐PC)', 'Not+D20421:D20432Challenging', ' Challenging', 'Undetermined (NC‐PC) ', 'Undetermined (PC-CH)',
                                                                            'Not Challenging ', 'Potentially Challenging '],
                                                      ['Potentially Challenging', 'Potentially Challenging', 'Potentially Challenging',
                                                      'Not Challenging', 'Not Challenging', 
                                                      'Undetermined (NC‐PC)', 'Undetermined (PC‐CH)', 'Undetermined (PC‐CH)', 'Undetermined (NC‐PC)',
                                                      'Undetermined (NC‐PC)', 'Not Challenging', 'Challenging', 'Undetermined (NC‐PC)', 'Undetermined (PC‐CH)',
                                                      'Not Challenging', 'Potentially Challenging'])

EPRI_up['Disposition_FES'] = EPRI_up['Disposition_FES'].replace(['Not Challenging/Undeter', '12/2/2011', 'Undetermined'],
                                                               np.nan)
EPRI_up['Disposition_FES'] = EPRI_up['Disposition_FES'].replace(['Undetermined (NC-PC)'],
                                                               'Undetermined (NC‐PC)')

#EPRI_up['Disposition_FES'].value_counts()

**Fire Type**

In [ ]:
EPRI_up['Fire Type'].unique()

In [ ]:
EPRI_up['Fire Type'] = EPRI_up['Fire Type'].replace(['laming combustion – external to component', 
                                                     'Flamingcombustion–externaltocomponent',
                                                    ': Flaming combustion – external to component',
                                                    'Flaming combustion – external to component '], 'Flaming combustion – external to component')

EPRI_up['Fire Type'] = EPRI_up['Fire Type'].replace(['Smolderingcombustion–internaltocomponent',
                                                    'Smoldering combustion – internal to componen'],
                                                    'Smoldering combustion – internal to component')

EPRI_up['Fire Type'] = EPRI_up['Fire Type'].replace(['Smolderingcombustion–externaltocomponent'],
                                                    'Smoldering combustion – external to component')

EPRI_up['Fire Type'] = EPRI_up['Fire Type'].replace(['Flamingcombustion–internaltocomponent',
                                                    'Flaming combustion – internal to componen',
                                                    ': Flaming combustion – internal to component'], 
                                                    'Flaming combustion – internal to component')
EPRI_up['Fire Type'] = EPRI_up['Fire Type'].replace(['Fire not observed and fire type indeterminate from post‐inspectio',
                                                    'Firenotobservedandfiretypeindeterminatefrompost‐inspectio',
                                                    'Firenotobservedandfiretypeindeterminatefrompost‐inspectio\nFireTypeUnknown:',
                                                    ' Fire not observed and fire type indeterminate from post‐inspectio',
                                                    'Fire not observed and fire type indeterminate from post‐inspectio\n',
                                                    'Fire not observed and fire type indeterminate from post-inspection'], 
                                                    'Fire not observed and fire type indeterminate from post‐inspection')
EPRI_up['Fire Type'] = EPRI_up['Fire Type'].replace(['Arc/electricdischarge',
                                                    ': Arc/electric discharge',
                                                    ' Arc/electric discharge'],
                                                   'Arc/electric discharge')

EPRI_up['Fire Type'] = EPRI_up['Fire Type'].replace(['No Fire\xa0‐\xa0False actuation of detection device',
                                                    'NoFire‐Falseactuationofdetectiondevice'],
                                                   'No Fire ‐ False actuation of detection device')

EPRI_up['Fire Type'] = EPRI_up['Fire Type'].replace(['Overheating – no  or flaming combustion',
                                                    'Overheating–nosmolderingorflamingcombustion',
                                                    'verheating – no smoldering or flaming combustion'],
                                                    'Overheating – no smoldering or flaming combustion')

EPRI_up['Fire Type'] = EPRI_up['Fire Type'].replace(['Other(specify)',
                                                    'Other (specify', 'Mechanicalequipmentmalfunction/failure'],
                                                   ['Other (specify)', 
                                                    'Other (specify)', 'Mechanical equipment malfunction/failure'])

EPRI_up['Fire Type'] = EPRI_up['Fire Type'].replace(['Mechanical equipment malfunction/failure',
                                                     'Overheated Material (lube oil, pump packing, thermal insulation, etc.)',
                                                     'Other electrical or electronic equipment', 
                                                     'Hot work (cutting/welding/grinding/etc.)', 
                                                     'Light smoke coming from ignition source\xa0‐\xa0minor or no visibility reduction in vicinity of fire', 
                                                     'Temporary electrical wiring or equipment'],
                                                   np.nan)

In [ ]:
EPRI_up['Fire Type'].value_counts()

**Fire Cause**

In [ ]:
EPRI_up['Fire Cause'].unique()

In [ ]:
EPRI_up['Fire Cause'] = EPRI_up['Fire Cause'].replace(['High Energy Arc Fault', 'HighEnergyArcFault(HEAF)',
                                                      'High Energy Arc Fault (HEAF'],
                                                     'High Energy Arc Fault (HEAF)')

EPRI_up['Fire Cause'] = EPRI_up['Fire Cause'].replace(['Electricalarcingorsparks(non‐HEAF)', 'Electrical arcing or sparks (non‐HEAF',
                                                      'Electrical arcing or sparks (non‐HEAF)\n'],
                                                     'Electrical arcing or sparks (non‐HEAF)')

EPRI_up['Fire Cause'] = EPRI_up['Fire Cause'].replace(['Electricalfailureresultinginoverheatingmaterials',
                                                      ':Electricalfailureresultinginoverheatingmaterials',
                                                      ':Electricalfailureresultinginoverheatingmaterials',
                                                      'vElectrical failure resulting in overheating materials',
                                                      ': Electrical failure resulting in overheating materials',
                                                      'Electrical failure resulting in overheating materials\n',
                                                      ' Electrical failure resulting in overheating materials'],
                                                     'Electrical failure resulting in overheating materials')

EPRI_up['Fire Cause'] = EPRI_up['Fire Cause'].replace(['OverheatedMaterial(lubeoil,pumppacking,thermalinsulation,etc.)',
                                                      ' Overheated Material (lube oil, pump packing, thermal insulation, etc.)',
                                                      ' Overheated Material (lube oil, pump packing, thermal insulation, etc',
                                                      'Overheated Material (lube oil, pump packing, thermal insulation, etc.'],
                                                     'Overheated Material (lube oil, pump packing, thermal insulation, etc.)')

EPRI_up['Fire Cause'] = EPRI_up['Fire Cause'].replace(['Hotwork(cutting/welding/grinding/etc.)',
                                                      'Hot work (cutting/welding/grinding/etc',
                                                      'Hot work (cutting/welding/grinding/etc.'],
                                                     'Hot work (cutting/welding/grinding/etc.)')

EPRI_up['Fire Cause'] = EPRI_up['Fire Cause'].replace(['Other(otherpersonnelerror,naturaleffect,etc.specifyincomments)',
                                                      'Other (personnel error, natural effect, etc. specify in comments)',
                                                      'Other (other personnel error, natural effect, etc. specify in comments'],
                                                     'Other (other personnel error, natural effect, etc. specify in comments)')

EPRI_up['Fire Cause'] = EPRI_up['Fire Cause'].replace(['Explosion (hydrogen gas ignition, fuel vapor ignition, other volatile fluid vapor ignition）',
                                                      'Explosion (hydrogen gas ignition, fuel vapor ignition)'],
                                                     'Explosion (hydrogen gas ignition, fuel vapor ignition, other volatile fluid vapor ignition)')

EPRI_up['Fire Cause'] = EPRI_up['Fire Cause'].replace(['Personnelerrorduringtestandmaintenanceactivity', 'Unkno', 
                                                       'Mechanicalequipmentmalfunction/failure', 'Mechanical malfunction/failure',
                                                      'Misuse of heating devices'],
                                                      ['Personnel error during test and maintenance activity', 'Unknown', 
                                                       'Mechanical equipment malfunction/failure', 'Mechanical equipment malfunction/failure',
                                                      'Personnel error: Misuse of heating devices'])

EPRI_up['Fire Cause'] = EPRI_up['Fire Cause'].replace(['Flamingcombustion–externaltocomponent', 'Not Challenging',
                                                       'In‐Situ', 'Potentially Challenging', 'Transient',
                                                       'Trash (i.e., solid refuse collected for disposal)', '', 'Arc/electricdischarge',
                                                                  'Electrical malfunction/failure', 'Electrical Failure (overheating, spark, HEAF)'],
                                                     np.nan)
EPRI_up['Fire Cause'] = EPRI_up['Fire Cause'].replace(['Electrical arcing or sparks (non‐HEAF)'],
                                                     'Electrical arcing or sparks (non-HEAF)')

In [ ]:
EPRI_up['Fire Cause'].value_counts()

**Building**

In [ ]:
EPRI_up['Building'].unique()

In [ ]:
EPRI_up['Building'] = EPRI_up['Building'].replace(['Main Transformer or Switch Yard ',
                                                              'Main\xa0Transformer\xa0or\xa0Switch\xa0Yard ',
                                                              'Main\xa0Transformer\xa0or\xa0Switch\xa0Yard',
                                                              ' Main Transformer or Switch Yard'],
                                                             'Main Transformer or Switch Yard')

EPRI_up['Building'] = EPRI_up['Building'].replace(['Auxiliary\xa0Building\xa0(PWR) ',
                                                              'Auxiliary\xa0Building\xa0(PWR)',
                                                              ' Auxiliary Building (PWR)'],
                                                             'Auxiliary Building (PWR)')

EPRI_up['Building'] = EPRI_up['Building'].replace(['Turbine\xa0Building ',
                                                              ' Turbine\xa0Building ',
                                                              'Turbine\xa0Building',
                                                              ' Turbine Building'],
                                                             'Turbine Building')

EPRI_up['Building'] = EPRI_up['Building'].replace(['Reactor\xa0Building\xa0(BWR) ',
                                                              'Reactor\xa0Building\xa0(BWR)',
                                                              ' Reactor Building (BWR)',
                                                              ' Reactor\xa0Building\xa0(BWR) '],
                                                             'Reactor Building (BWR)')

EPRI_up['Building'] = EPRI_up['Building'].replace(['Radwaste\xa0Building ',
                                                              'Radwaste\xa0Building',
                                                              ' Radwaste\xa0Building',
                                                              ' Radwaste Building'],
                                                             'Radwaste Building')

EPRI_up['Building'] = EPRI_up['Building'].replace(['Other (Specify in comments) ',
                                                              'Other\xa0(Specify\xa0in\xa0comments) ',
                                                              ' Other\xa0(Specify\xa0in\xa0comments) ',
                                                              ': Other (Specify in comments)',
                                                              'Other\xa0(Specify\xa0in\xa0comments)',
                                                              ' Other (Specify in comments)',
                                                              'Other (Specify in comment', 'Other\xa0(Specify\xa0in\xa0comments',
                                                  'Other (specify in comments)'],
                                                             'Other (Specify in comments)')

EPRI_up['Building'] = EPRI_up['Building'].replace(['Diesel\xa0Generator\xa0Building ',
                                                              'Diesel\xa0Generator\xa0Building',
                                                               'Diesel\xa0Generator\xa0Buildin',
                                                              ' Diesel Generator Building'],
                                                             'Diesel Generator Building')

EPRI_up['Building'] = EPRI_up['Building'].replace(['Circulating\xa0Water\xa0Pump\xa0house/Intake\xa0Structure ',
                                                              'Circulating\xa0Water\xa0Pump\xa0house/Intake\xa0Structure',
                                                              ' Circulating Water Pump house/Intake Structure'],
                                                             'Circulating Water Pump house/Intake Structure')

EPRI_up['Building'] = EPRI_up['Building'].replace(['Containment\xa0(PWR) ',
                                                              ' Containment (PWR)'],
                                                             'Containment (PWR)')

EPRI_up['Building'] = EPRI_up['Building'].replace(['Drywell\xa0(BWR) ',
                                                              ' Drywell (BWR)'],
                                                             'Drywell (BWR)')

EPRI_up['Building'] = EPRI_up['Building'].replace(['Control\xa0building ',
                                                              ' Control building'],
                                                             'Control building')

EPRI_up['Building'] = EPRI_up['Building'].replace(['Service\xa0Water\xa0Pump\xa0house ',
                                                              'Main Control Room'],
                                                              ['Service Water Pump house',
                                                              np.nan])

EPRI_up['Building'] = EPRI_up['Building'].replace(['Potentially Challenging', ' ',
                                                               'Not Challenging', ' Main\xa0Control\xa0Room'],
                                                             np.nan)

EPRI_up['Building'] = EPRI_up['Building'].replace(['Turbine Building, Other (specify'],
                                                 'Turbine Building')

EPRI_up['Building'] = EPRI_up['Building'].replace(['Challenging', 'Battery Room',
                                                  'Sewage Treatment', 'Warehouse'], np.nan)

EPRI_up['Building'] = EPRI_up['Building'].replace(['Circulating Water Pump house/'],
                                                  'Circulating Water Pump house/Intake Structure')

EPRI_up['Building'] = EPRI_up['Building'].replace(['Main Transformer or Switch Yar'],
                                                  'Main Transformer or Switch Yard')

EPRI_up['Building'] = EPRI_up['Building'].replace(['Main Transformer or Switch Yar'],
                                                  'Main Transformer or Switch Yard')

EPRI_up['Building'].value_counts()

**Combustible Group**

In [ ]:
EPRI_up['Combustible Group'].unique()

In [ ]:
EPRI_up['Combustible Group'] = EPRI_up['Combustible Group'].replace(['Other (other personnel error, natural effect, etc. specify in comments)', 
                                                         'Other electrical or electronic equipment', 'Hot work (cutting/welding/grinding/etc.)',
                                                         'Mechanical equipment malfunction/failure', '3/19/2001', '3/19/2002', 'Electrical arcing or sparks (non‐HEAF)',
                                                         'Overheating – no smoldering or flaming combustion',],
                                                        np.nan) 

EPRI_up['Combustible Group'] = EPRI_up['Combustible Group'].replace(['n‐Situ', 'Unkno'],
                                                       ['In‐Situ', 'Unknown'])

EPRI_up['Combustible Group'] = EPRI_up['Combustible Group'].replace(['Transient(Gas)', 'Transient (Gas'],
                                                       'Transient (Gas)')

EPRI_up['Combustible Group'] = EPRI_up['Combustible Group'].replace(['In‐Situ', 'In-Situ', 'In Situ'], 'In_Situ')

EPRI_up['Combustible Group'] = EPRI_up['Combustible Group'].replace('Transient (Gas)', 'Transient_Gas')

EPRI_up['Combustible Group'] = EPRI_up['Combustible Group'].replace('Transient (Liquid)', 'Transient_Liquid')

EPRI_up['Combustible Group'] = EPRI_up['Combustible Group'].replace(['nsulation'], 'Insulation')

# EPRI_up['Combustible Group'].value_counts()

**Combustible Group Type**

In [ ]:
EPRI_up['Type'].unique()

In [ ]:
EPRI_up['Type'] = EPRI_up['Type'].replace(['LubeOilorgrease',
                                                      'Otherelectricalorelectronicequipmen',
                                                      'Otherelectricalorelectronicequipment',
                                                      'Temporaryelectricalwiringorequipment',
                                                      'Thermalinsulationmaterials',
                                                      'Trash(i.e.,solidrefusecollectedfordisposal)',
                                                      'Temporarythermalinsulationmaterials',
                                                      'Cleaningsolventsorpaint',
                                                      'Cellulosicmaterialsincludingwood,paperorothersolidtransients',
                                                      'Cellulosic materials including wood, paper or other solid transients\nForm\nN',
                                                      'Structuralcomponen',
                                                      'Structuralcomponen',
                                                       'Structuralcomponent',
                                                       'Other(specif', 'Other(specify)', 'Other(specify', 'Other (specify',
                                                       'Trash (i.e., solid refuse collected for disposal',
                                                       'Typenotspecified',
                                          'Cellulosic materials including wood, paper or ot',
                                          'Thermal insulation materials',
                                          'Other Transient Combustible (specify)',
                                          'Other (specify)',
                                          'Other (specify)'],
                                                     ['Lube Oil or grease',
                                                     'Other electrical or electronic equipment',
                                                     'Other electrical or electronic equipment',
                                                     'Temporary electrical wiring or equipment',
                                                     'Thermal insulation materials',
                                                     'Trash (i.e., solid refuse collected for disposal)',
                                                     'Temporary thermal insulation materials',
                                                     'Cleaning solvents or paint',
                                                     'Cellulosic materials including wood, paper or other solid transients',
                                                     'Cellulosic materials including wood, paper or other solid transients',
                                                     'Structural component',
                                                     'Structural component',
                                                      'Structural component', 'Other (specify)', 'Other (specify)', 'Other (specify)', 'Other (specify)',
                                                      'Trash (i.e., solid refuse collected for disposal)',
                                                      'Type not specified',
                                                     'Cellulosic materials including wood, paper or other solid transients',
                                                     'Temporary thermal insulation materials',
                                                     'Other (Specify)',
                                                     'Other (Specify)',
                                                     'Other (Specify)'])

EPRI_up['Type'] = EPRI_up['Type'].replace(['Other electrical or electronic equipmentOther electrical or electronic equipment',
                                                      'Other electrical or electronic equipment\n',
                                                      'Other electrical or electronic euiment',
                                                      ],
                                                     'Other electrical or electronic equipment')

EPRI_up['Type'] = EPRI_up['Type'].replace(['Flaming combustion – external to component',
                                                      'Transient', 'Liquid',
                                                      'Confined to the object of origin (localized/single subcomponent)',
                                                      'No damage', 'Temporary ',
                                                      'Fire not observed and fire type indeterminate from post‐inspecti',
                                                      'Hot work (cutting/welding/grinding/etc.)', 'In‐Situ',
                                          'Multiple Cables Not In Tray or Bundle',
                                          'Single Cable Tray',
                                          'Cable Tray Stack', 'Unknown'],
                                                     np.nan)

EPRI_up['Type'] = EPRI_up['Type'].replace(['Lube Oil or grease'],
                                         'Lube Oil or Grease')

EPRI_up['Type'] = EPRI_up['Type'].replace(['Other (specify)'],
                                         'Other (Specify)')

EPRI_up['Type'].value_counts()

**Combustible Group Form**

In [ ]:
EPRI_up['Form'].unique()

In [ ]:
EPRI_up['Form'] = EPRI_up['Form'].str.replace(r'\xa0', ' ')

EPRI_up['Form'] = EPRI_up['Form'].replace(['Leak‐OilsoakedInsulation'],
                                                     'Leak ‐ Oil soaked Insulation')

EPRI_up['Form'] = EPRI_up['Form'].replace(['MultipleCablesNotInTrayorBundle'],
                                                     'Multiple Cables Not In Tray or Bundle')

EPRI_up['Form'] = EPRI_up['Form'].replace(['Other (specify)', 'Other(Specify)',
                                                      'Other(specify)', 'Other (specify',
                                          'Other (Specify'],
                                                     'Other (Specify)')

EPRI_up['Form'] = EPRI_up['Form'].replace(['Unconfinedspill', 'Pressurizedinacontainer',
                                                      'Jetfromapressurizedsource', 'Spillconfinedbycurbsorotherphysicalfeat',
                                                      'Containedwithincomponent', 'Containedwithinreservoir', 'SingleCable',
                                          ],
                                                     ['Unconfined spill', 'Pressurized in a container',
                                                     'Jet from a pressurized source', 'Spill confined by curbs or other physical feat',
                                                     'Contained within component', 'Contained within reservoir', 'Single Cable'])

EPRI_up['Form'] = EPRI_up['Form'].replace(['Flaming combustion – external to component', 'Contained within'],
                                                      np.nan)

EPRI_up['Form'] = EPRI_up['Form'].replace(['NA, Insulation Multiple Bundles'],
                                         'Multiple Bundles')

EPRI_up['Form'] = EPRI_up['Form'].replace(['Leak ‐ Oil soaked Insulation', 'Leak - Oil soaked'],
                                         'Leak - Oil soaked Insulation')

EPRI_up['Form'].value_counts()

**System**

In [ ]:
EPRI_up['System'].unique()

In [ ]:
EPRI_up['System'] = EPRI_up['System'].replace([' Switchyard\xa0System', 'Switchyard\xa0System',
                                                          ' Switchyard System'],
                                                         'Switchyard System')

EPRI_up['System'] = EPRI_up['System'].replace([' Low\xa0Voltage\xa0Power\xa0System\xa0(600V\xa0and\xa0less)',
                                                          'Low\xa0Voltage\xa0Power\xa0System\xa0(600V\xa0and\xa0less)',
                                                          ' Low Voltage Power System (600V and less)',
                                                          ' Low Voltage Power System (600V and less',
                                                          'Low Voltage Power System (600V and less'],
                                                         'Low Voltage Power System (600V and less)')
EPRI_up['System'] = EPRI_up['System'].str.replace(r'\xa0', ' ')

EPRI_up['System'] = EPRI_up['System'].replace([' General Ventillation Systems',' Non Essential Service Water System',
                                                           ' Essential Service Water System', ' Medium Voltage Power System (601V to 35kV)',
                                                           ' Reactor Recirculation System (BWR)', ' Other ', ' Medium Voltage Power System, Class 1E',
                                                           ' Feedwater System', ' DC Power System', 'Transformers ',
                                                          ' High Pressure Safety Injection System (PWR)', ' Emergency Onsite Power Supply System',
                                                          'Electric Motor', 'Pumps', 'Undetermined (PC‐CH)',
                                                          'Fire in the Service Building.', 'Electrical panel'],
                                                         ['General Ventillation Systems', 'Non Essential Service Water System',
                                                         'Essential Service Water System', 'Medium Voltage Power System (601V to 35kV)',
                                                         'Reactor Recirculation System (BWR)','Other', 'Medium Voltage Power System, Class 1E',
                                                         'Feedwater System', 'DC Power System', np.nan,
                                                         'High Pressure Safety Injection System (PWR)', 'Emergency Onsite Power Supply System',
                                                         np.nan, np.nan, np.nan, np.nan, np.nan])

EPRI_up['System'].value_counts()

**Component Group**

In [ ]:
EPRI_up['Component Group'].unique()

In [ ]:
EPRI_up['Component Group'] = EPRI_up['Component Group'].str.replace(r'\xa0', ' ')

EPRI_up['Component Group'] = EPRI_up['Component Group'].replace(['Transformers ', 'Pumps ', ' Pumps ', 'Pump',
                                                                            ' Pumps', ' Transformers ',
                                                                            'Electric Motor ', 'Air Compressors ', ' Electric Motor ',
                                                                            ' Batteries ', 'Boilers ', 'Lighting Ballasts ', 'Outlets ',
                                                                            '   Other', ' Outlets', '  Junction Boxes', '  Lighting Ballasts',
                                                                             '  Generator', '  Cable/Wiring', '   Transformers',
                                                                            ' Other ', ' Transformers', ' Cable/Wiring ', ' Electrical panel ',
                                                                            'Crane ', 'Cable/Wiring ', 'Batteries ', 'Generator ',
                                                                            '  Inverter', ' Junction Boxes', '  Transformers', ' Fans', ' Lighting Ballasts',
                                                                            'Low Voltage Power System (600V and less)', ' Batteries',
                                                                            '  Electrical panel', ' Electrical panel', 'Electrical pane',
                                                                            '  Electric Motor', ' Cable/Wiring', ' Air Compressors',
                                                                            ' Electric Motor', ' Generator', ' Crane',
                                                                            'Electrical panel '],
                                                                           ['Transformers', 'Pumps', 'Pumps', 'Pumps',
                                                                           'Pumps', 'Transformers',
                                                                           'Electric Motor', 'Air Compressors', 'Electric Motor',
                                                                           'Batteries', 'Boilers', 'Lighting Ballasts', 'Outlets',
                                                                           'Other', 'Outlets', 'Junction Boxes', 'Lighting Ballasts',
                                                                            'Generator', 'Cable/Wiring', 'Transformers',
                                                                            'Other', 'Transformers', 'Cable/Wiring', 'Electrical panel',
                                                                           'Crane', 'Cable/Wiring', 'Batteries', 'Generator',
                                                                           'Inverter', 'Junction Boxes', 'Transformers', np.nan, 'Lighting Ballasts',
                                                                           np.nan, 'Batteries',
                                                                           'Electrical panel', 'Electrical panel', 'Electrical panel',
                                                                           'Electric Motor', 'Cable/Wiring', 'Air Compressors',
                                                                           'Electric Motor', 'Generator', 'Crane',
                                                                           'Electrical panel'])

EPRI_up['Component Group'] = EPRI_up['Component Group'].replace(['Other ', ' Other (Capacitor Voltage, Poten',
                                                                             ' Other', '  Other',
                                                                            'Other (Specify in comments)'],
                                                                           'Other')

EPRI_up['Component Group'] = EPRI_up['Component Group'].replace(['Lube Oil ', ' Lube Oil'],
                                                                           'Lube Oil')

EPRI_up['Component Group'] = EPRI_up['Component Group'].replace([' Electric motor driven pumps',
                                                                            'Motor control center',
                                                                            'General Ventillation Systems',
                                                                            'Emergency Onsite Power Supply System'],
                                                                           np.nan)
EPRI_up['Component Group'] = EPRI_up['Component Group'].replace(['Main/Reheat Steam System', 'Inverter',
                                                                'Yard Handling and Maintenance System', 'Electric heaters'],
                                                               np.nan)


#  EPRI_up['Component Group'].value_counts()

**Component**

In [ ]:
EPRI_up['Component'].unique()

In [ ]:
EPRI_up['Component'] = EPRI_up['Component'].str.replace(r'\xa0', ' ')

In [ ]:
EPRI_up['Component'] = EPRI_up['Component'].replace([' Fans', 'Other ', 'Inverter ',
                                                     '  Motor control center', ' Motor control center ',
                                                    'Fans ', ' Switchgear ', ': Inverter',
                                                    ' Elevator motor ', 'Elevator motor ',
                                                    ' Elevator motor',
                                                    ' Fans ', ' Crane motor', ' Inverter ', 'MG set ',
                                                    ' Load center', ' MG set', 'Relay rack ',
                                                    ' Crane motor ', 'Motor control center ', 'Battery charger ',
                                                    ' Inverter', '  Relay rack', '  Diesel driven pumps',
                                                    '  Main turbine generator', ' Dry transformer', ' Relay rack',
                                                    ' Main turbine generator', ' Motor control center',
                                                    'Diesel driven pumps ', 'Dry transformer ',
                                                    'Wall Mounted or Sealed ', ' Wall Mounted or Sealed',
                                                    'Electric Bus Bar or Bus Duct', ' Electric bus bar or bus duct',
                                                    ' Turbine‐driven pump', 'Turbine-driven pump',
                                                    ' Diesel driven pumps',
                                                    ' Motor operated valve ', 'Main turbine generator '],
                                                   ['Fans', 'Other', 'Inverter',
                                                   'Motor control center', 'Motor control center',
                                                   'Fans', 'Switchgear', 'Inverter',
                                                   'Elevator motor', 'Elevator motor',
                                                   'Elevator motor',
                                                   'Fans', 'Crane motor', 'Inverter', 'MG set',
                                                   'Load center', 'MG set', 'Relay rack',
                                                   'Crane motor', 'Motor control center', 'Battery charger',
                                                   'Inverter', 'Relay rack', 'Diesel driven pumps',
                                                   'Main turbine generator', 'Dry transformer', 'Relay rack',
                                                   'Main turbine generator', 'Motor control center',
                                                   'Diesel driven pumps', 'Dry transformer',
                                                   'Wall Mounted or Sealed', 'Wall Mounted or Sealed',
                                                   'Electric bus bar or bus duct', 'Electric bus bar or bus duct',
                                                   'Turbine‐driven pump', 'Turbine‐driven pump',
                                                   'Diesel driven pumps',
                                                   'Motor operated valve', 'Main turbine generator'])

In [ ]:
EPRI_up['Component'] = EPRI_up['Component'].replace([' Class 1A emergency generator ',
                                                    'Class 1A emergency generator ',
                                                    '  Class 1A emergency generator',
                                                    ' Class 1A emergency generator',
                                                    '   Class 1A emergency generator'],
                                                   'Class 1A emergency generator')

In [ ]:
EPRI_up['Component'] = EPRI_up['Component'].replace(['Other (Capacitor Voltage, Poten ',
                                                    ' Other (Capacitor Voltage, Poten ',
                                                     ' Other (Capacitor Voltage, Poten',
                                                    '  Other (Capacitor Voltage, Poten',
                                                    'Other (Capacitor Voltage, Potential',
                                                    'Other (Capacitor Voltage, Poten'],
                                                   'Other (capacitor voltage, potential, etc.)')

In [ ]:
EPRI_up['Component'] = EPRI_up['Component'].replace([' Other (specify in comments) ',
                                                    'Other (specify in comments) ',
                                                    '  Other (specify in comments)',
                                                    ' Other (specify in comments)',
                                                    'Other'],
                                                   'Other (specify in comments)')

In [ ]:
EPRI_up['Component'] = EPRI_up['Component'].replace([' Circuit breaker or other distribu ',
                                                    'Circuit breaker or other distribu ',
                                                    ' Circuit breaker or other distribu',
                                                    '  Circuit breaker or other distribu',
                                                    '  Circuit breaker or other distribu',
                                                    'Circuit breaker or other distributio',
                                                    '  Circuit breaker or other distribu',
                                                    'Circuit breaker or other distribu'],
                                                   'Circuit breaker or other distribution panel')

In [ ]:
EPRI_up['Component'] = EPRI_up['Component'].replace([' Oil‐filled Transformer ',
                                                    'Oil‐filled Transformer ',
                                                    'Oil-filled Transformer'],
                                                   'Oil‐filled Transformer')

In [ ]:
EPRI_up['Component'] = EPRI_up['Component'].replace(['Electric motor driven pumps ', 
                                                     ' Electric motor driven pumps ', 
                                                     ' Electric motor driven pumps',
                                                    '  Electric motor driven pumps',
                                                    '   Electric motor driven pumps'],
                                                   'Electric motor driven pumps')

In [ ]:
EPRI_up['Component'] = EPRI_up['Component'].replace(['Other instrument/control cabin',
                                                    ' Other instrument/control cabin ',
                                                    ' Other instrument/control cabin',
                                                    'Other instrument/control cabin ',
                                                    ' Other instrument/control cabin',
                                                    '  Other instrument/control cabin',
                                                    'Other instrument/control cabinet'],
                                                   'Other instrument/control cabinet')

In [ ]:
EPRI_up['Component'] = EPRI_up['Component'].replace([' Main control board (i.e., the ma ',
                                                    'Main control board (i.e., the ma ',
                                                    'Main control board (i.e., the main'],
                                                   'Main control board (i.e., the ma')

In [ ]:
EPRI_up['Component'] = EPRI_up['Component'].replace(['Other generator (specify in com ', 
                                                     'Other generator (specify in com', 
                                                     ' Other generator (specify in com',
                                                     'Other generator (specify in comme'],
                                                   'Other generator (specify in comments)')

In [ ]:
EPRI_up['Component'].value_counts().sort_index(ascending=True)

**Mode Prior**

In [ ]:
EPRI_up['Mode Prior'].unique()

In [ ]:
EPRI_up['Mode Prior'] = EPRI_up['Mode Prior'].str.replace(r'\xa0', ' ')

EPRI_up['Mode Prior'] = EPRI_up['Mode Prior'].replace([' Refueling', 'Refueling ',
                                                                   ' Refueling ', 'efueling', 'Refueling M'],
                                                                 'Refueling')

EPRI_up['Mode Prior'] = EPRI_up['Mode Prior'].replace(['Operate ', ' Operate ', ' Operate',
                                                                   ': Operate', 'Operat', 'Operate M'],
                                                                 'Operate')

EPRI_up['Mode Prior'] = EPRI_up['Mode Prior'].replace(['Cold Shutdown M',
                                                                  'Cold Shutdown ', ' Cold Shutdown',
                                                                  ': Cold Shutdown'],
                                                                 'Cold Shutdown')

EPRI_up['Mode Prior'] = EPRI_up['Mode Prior'].replace(['Unknown ', '5/5/2006', 
                                                                   'Undetermined (NC‐PC) R', 'Preliminary'],
                                                                 ['Unknown', np.nan, np.nan, np.nan])

# EPRI_up['Mode Prior'].value_counts()

**Mode After**

In [ ]:
EPRI_up['Mode After'].unique()

In [ ]:
EPRI_up['Mode After'] = EPRI_up['Mode After'].str.replace(r'\xa0', ' ')

EPRI_up['Mode After'] = EPRI_up['Mode After'].replace(['Refueling ', ' Refueling',
                                                                       ' Refueling ', ': Refueling',
                                                                      ': Refueling P', 'Refuelin', 'Refueling P'],
                                                                     'Refueling')

EPRI_up['Mode After'] = EPRI_up['Mode After'].replace(['Operate ', ' Operate ',
                                                                  ' Operate', ': Operate'
                                                                  'Operate P', 'Operat',
                                                                  ': Operate', 'Operate P'],
                                                                 'Operate')

EPRI_up['Mode After'] = EPRI_up['Mode After'].replace(['Cold Shutdown ', ' Cold Shutdown', ': Cold Shutdown'],
                                                                 'Cold Shutdown')

EPRI_up['Mode After'] = EPRI_up['Mode After'].replace([': Hot Standby', ' Hot Standby0'],
                                                                 'Hot Standby')

EPRI_up['Mode After'] = EPRI_up['Mode After'].replace(['Unknown ', ' Unknown', ': Unknown'],
                                                                 'Unknown')

EPRI_up['Mode After'] = EPRI_up['Mode After'].replace(['5/5/2006', 'Startup/Hot Sta', 'Hot Shutdown P'],
                                                                 [np.nan, np.nan, 'Hot Shutdown'])

EPRI_up['Mode After'].value_counts()

**Plant Area**

In [ ]:
EPRI_up['Plant Area'].unique()

In [ ]:
EPRI_up['Plant Area'] = EPRI_up['Plant Area'].replace(['Switchgear\xa0Room', ' Switchgear\xa0Room',
                                                                  ' Switchgear Room'], 
                                                                  'Switchgear Room')

EPRI_up['Plant Area'] = EPRI_up['Plant Area'].replace(['Battery\xa0Room',
                                                                  ' Battery\xa0Room'],
                                                                 'Battery Room')

EPRI_up['Plant Area'] = EPRI_up['Plant Area'].replace(['Cable\xa0Spreading\xa0Room'],
                                                                 'Cable Spreading Room')

EPRI_up['Plant Area'] = EPRI_up['Plant Area'].replace([' Main\xa0Control\xa0Room',
                                                                  'Main\xa0Control\xa0Room',
                                                                  ' Main Control Room'],
                                                                 'Main Control Room')

EPRI_up['Plant Area'] = EPRI_up['Plant Area'].replace([' Fuel\xa0handling\xa0area', ' Fuel handling area'],
                                                                 'Fuel handling area')

EPRI_up['Plant Area'] = EPRI_up['Plant Area'].replace([' Cable\xa0vault/tunnel\xa0area',
                                                                  ' Cable vault/tunnel area'],
                                                                 'Cable vault/tunnel area')

EPRI_up['Plant Area'] = EPRI_up['Plant Area'].replace(['Potentially Challenging', ' Containment/Drywell'],
                                                                  [np.nan, 'Containment/Drywell'])

EPRI_up['Plant Area'].value_counts()

**Detection Performance**

In [ ]:
EPRI_up['Detection Performance'].unique()

In [ ]:
EPRI_up['Detection Performance'] = EPRI_up['Detection Performance'].str.replace(r'\xa0', ' ')

In [ ]:
EPRI_up['Detection Performance'] = EPRI_up['Detection Performance'].replace(['Plant area had installed detection which did not actuate, fire was detected visually or by other means–no indication of system failure',
                                                                            'Plant area had installed detection which did not，actuate, fire was detected visually or by other，means – no indication of system failure',
                                                                            'Plant area had installed detection which did not actuate, fire was detected visually or by other，means – no indication of system failure',
                                                                            'Plant area had installed detection which did not actuate, fire was detected visually or by other，means – no indication of system failure',
                                                                            'Plant area had installed detection which did not,actuate, fire was detected visually or by other means – no indication of system failure',
                                                                            'Plant area had installed detection which did no actuate, fire was detected visually or by other means – no indication of system failure',
                                                                            'Plant area had installed detection which did not actuate, fire was detected visually or by other means – no indication of system failure',
                                                                            'Plant area had installed detection which did not actuate, fire was detected visually or by other means – no indication of system failure system failure'],
                                                                           'Plant area had installed detection which did not actuate, fire was detected visually or by other，means – no indication of system failure')

In [ ]:
EPRI_up['Detection Performance'] = EPRI_up['Detection Performance'].replace([' Plant area did not have installed detection capability',
                                                                            'Plant area did not have installed detection，capability'],
                                                                           'Plant area did not have installed detection capability')

In [ ]:
EPRI_up['Detection Performance'] = EPRI_up['Detection Performance'].replace([' Plant area had installed detection which actuated'],
                                                                           'Plant area had installed detection which actuated')

In [ ]:
EPRI_up['Detection Performance'] = EPRI_up['Detection Performance'].replace(['Unknow', ' Unknown'],
                                                                           'Unknown')

In [ ]:
EPRI_up['Detection Performance'] = EPRI_up['Detection Performance'].replace(['Self‐extinguished (no plant staff intervention)',
                                                                            'Fuel supply removed (simple manual action e.g valve turn off)',
                                                                            'Hose stream or hose reel,Multiple portable fire extinguishers'],
                                                                           np.nan)

**New Disposition**

In [ ]:
EPRI_up['Disposition_new'] = EPRI_up['Disposition_FES']

In [ ]:
EPRI_up['Disposition_new'] = EPRI_up['Disposition_new'].replace(['Challenging', 'Potentially Challenging', 'Undetermined (PC‐CH)'],
                                                                     'Challenging')

EPRI_up['Disposition_new'] = EPRI_up['Disposition_new'].replace(['Not Challenging', 'Undetermined (NC‐PC)'],
                                                                     'Not Challenging')

EPRI_up['Disposition_new'] = EPRI_up['Disposition_new'].replace(['Undetermined', 'Not Evaluated'],
                                                                     'Not Evaluated')

# print(EPRI_up['Disposition_new'].unique())


# Save Data

In [ ]:
EPRI_up.to_excel(sys_up_data + 'EPRI_1990-2014.xlsx')